In [ ]:
import pandas as pd
import pickle

# REMOVED
#import torch
#import transformers
#from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
import sys
import csv
import re
from tqdm.notebook import tqdm
import sys
import csv
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
from src.preprocess import *
from src.models import *
from src.lib import *

## Explore data

### kinds of cause:
- 1 = No reason
- 2 = Bias or abuse
- 3 = Jobs and careers
- 4 = Medication
- 5 = Relationships
- 6 = Alienation

In [3]:
import numpy as np
import pandas as pd

## Training and Testing files
read_file_training = pd.read_csv('./data/added_CAMS_data.csv', encoding="ISO-8859-1")
read_file_testing = pd.read_csv('./data/IntentSDCNL_Testing.csv', encoding = "ISO-8859-1")



#training data
df_train=pd.DataFrame(read_file_training,columns= ['selftext','cause', 'inference'])### 
df_train = df_train.rename(
    columns={'selftext': 'text'})
df_train= df_train.convert_dtypes()

#testing data
df_test=pd.DataFrame(read_file_testing,columns= ['selftext','ANNOTATIONS', 'Interpretations'])#### 
df_test = df_test.rename(columns={'selftext': 'text', 'ANNOTATIONS': 'cause', 'Interpretations': 'inference'})
df_test= df_test.convert_dtypes()

## Sampling file
# df_samples=pd.read_csv('./data/sample.csv', encoding = "ISO-8859-1")

## add samples to training file

frame = [df_train]
df_train =  pd.concat(frame)

In [4]:
df_test.drop(df_test[df_test.text=="emptypost"].index, inplace=True)
df_test.drop(df_test[df_test.cause==" "].index, inplace=True)
df_test.drop(df_test[df_test.inference==" "].index, inplace=True)

# print(df_train)

df_train.drop(df_train[df_train.text=="emptypost"].index, inplace=True)
df_train.drop(df_train[df_train.cause==" "].index, inplace=True)
df_train.drop(df_train[df_train.inference==" "].index, inplace=True)

df_test.dropna(inplace=True)
df_train.dropna(inplace=True)

In [ ]:
df_train.info()

In [ ]:
import matplotlib.pyplot as plt
print("###### Class distribution of training data #####")
labels = ["0", "1", "2", "3", "4", "5"]
fig, ax = plt.subplots(1, 1, figsize=(5,5))
df_train['cause'].value_counts().plot.pie(autopct = '%1.1f%%', labels=labels)


In [ ]:

print("###### Class distribution of testing data #####")
labels = ["0", "1", "2", "3", "4", "5"]
fig, ax = plt.subplots(1, 1, figsize=(5,5))
df_test['cause'].value_counts().plot.pie(autopct = '%1.1f%%', labels=labels)


In [ ]:
df_test['cause'].value_counts()

In [ ]:
frame = [df_test, df_train]
data =  pd.concat(frame)
data

#### A Sample

In [ ]:
print_plot(df_train,0)

## Clean and preprocess the text

####  Clean Training and Testing data

In [11]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text)     
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text


#training data
df_train = df_train.reset_index(drop=True)
df_train['text'] = df_train['text'].apply(clean_text)
df_train['text'] = df_train['text'].apply(clean_text)

##testing data
df_train = df_train.reset_index(drop=True)
df_test['text'] = df_test['text'].apply(clean_text)
df_test['text'] = df_test['text'].str.replace('\d+', '')

#### A sample after cleanup

In [ ]:
print_plot(df_train,0)

#### Intialize parameters

In [13]:
# Define some Global Variables
max_nb_words = 60000
max_features = 10543 # Maximum Number of words we want to include in our dictionary
maxlen = 500 # No of words in question we want to create a sequence with
emb_dim = 100 # Size of word to vec embedding we are usingtokenizer = Tokenizer(num_words=max_nb_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)


#### Initial Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=max_nb_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['text'].values)# tokenizer on whole training and testing data
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X_train = tokenizer.texts_to_sequences(df_train['text'].values)
X_train = pad_sequences(X_train, maxlen=maxlen)
print('Shape of data tensor:', X_train.shape)

In [16]:
Y_train = pd.get_dummies(df_train['cause']).values
print('Shape of label tensor:', Y_train.shape)

Shape of label tensor: (2877, 6)


In [ ]:
X_test = tokenizer.texts_to_sequences(df_test['text'].values)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('Shape of data tensor:', X_test.shape)

In [ ]:
Y_test = pd.get_dummies(df_test['cause']).values
print('Shape of label tensor:', Y_test.shape)

### Finally some validation data from Training Data

In [19]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,random_state=0,test_size=0.1)

In [ ]:
print('TrainingData',X_train.shape,Y_train.shape)
print('TestingData',X_test.shape,Y_test.shape)
print('ValidationData',X_val.shape,Y_val.shape)


In [21]:
#print a test sample
#print('test example:', X_test[100])

In [22]:
 len(X_test[100])

500

#### Intialize Embedding

In [ ]:
embedding_vectors = {}
emb_dim = 100
with open(f'./embeddings/glove.6B.{emb_dim}d.txt','r',encoding='utf-8') as file:

    for row in file:
        values = row.split(' ')
        word = values[0]
        weights = np.asarray([float(val) for val in values[1:]])
        embedding_vectors[word] = weights
print(f"Size of vocabulary in GloVe: {len(embedding_vectors)}")   

In [ ]:
#initialize the embedding_matrix with zeros

if max_nb_words is not None: 
    vocab_len = max_nb_words
else:
    vocab_len = len(word_index)+1
embedding_matrix = np.zeros((vocab_len, emb_dim))
oov_count = 0
oov_words = []
for word, idx in word_index.items():
    if idx < vocab_len:
        embedding_vector = embedding_vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
        else:
            oov_count += 1 
            oov_words.append(word)
#Check some of the out of vocabulary words
print(f'A few out of valubulary words: {oov_words[0:5]}')

In [ ]:
print(f'{oov_count} out of {vocab_len} words were OOV.')

## NN models

In [27]:
##Global variables
batch_size = 256
epochs  = 20

#### a) LSTM Model

In [ ]:
lstm_model = LSTM_model(embedding_matrix,vocab_len,emb_dim)

##### Train the model

In [ ]:
%%time
history = lstm_model.fit(X_train, np.asarray(Y_train), validation_data=(X_val, np.asarray(Y_val)), batch_size = batch_size, epochs = epochs)

In [ ]:
#plot accuracy
plot_accuracy(history,epochs)

In [ ]:
# Accuracy
train_test_accuracy(lstm_model,X_train,Y_train,X_test,Y_test)

In [ ]:
#Classification report
report_nn(lstm_model,X_test,Y_test)

#### b) CNN

In [ ]:
cnn_model=CNN_model(embedding_matrix,vocab_len)

##### Train the model

In [ ]:
%%time
history = cnn_model.fit(X_train, np.asarray(Y_train), validation_data=(X_val, np.asarray(Y_val)),batch_size = batch_size, epochs = epochs)

In [ ]:
#plot accuracy
plot_accuracy(history,epochs)

In [ ]:
# Accuracy
train_test_accuracy(cnn_model,X_train,Y_train,X_test,Y_test)

In [ ]:
#Classification report
report_nn(cnn_model,X_test,Y_test)

#### d) GRU

In [ ]:
gru_model = GRU_model(embedding_matrix,vocab_len)

In [ ]:
%%time
history = gru_model.fit(X_train, np.asarray(Y_train), validation_data=(X_val, np.asarray(Y_val)), batch_size = batch_size, epochs = epochs)

In [ ]:
#plot accuracy
plot_accuracy(history,epochs)

In [ ]:
# Accuracy
train_test_accuracy(gru_model,X_train,Y_train,X_test,Y_test)

In [ ]:
#Classification report
report_nn(gru_model,X_test,Y_test)


#### e) CNN+GRU

In [ ]:
cnn_gru_model =CNN_GRU_model(embedding_matrix,vocab_len)

In [ ]:
%%time
history=cnn_gru_model.fit(X_train, np.asarray(Y_train), validation_data=(X_val, np.asarray(Y_val)),epochs=epochs, batch_size=batch_size)

In [ ]:
#plot accuracy
plot_accuracy(history,epochs)

In [ ]:
# Accuracy
train_test_accuracy(cnn_gru_model,X_train,Y_train,X_test,Y_test)

In [ ]:
#Classification report
report_nn(cnn_gru_model,X_test,Y_test)

#### f) CNN+LSTM

In [ ]:
cnn_lstm_model = CNN_LSTM_model(embedding_matrix,vocab_len)

In [ ]:
%%time
history=cnn_lstm_model.fit(X_train, np.asarray(Y_train), validation_data=(X_val, np.asarray(Y_val)),epochs=epochs, batch_size=batch_size)

In [ ]:
#plot accuracy
plot_accuracy(history, epochs)

In [ ]:
# Accuracy
train_test_accuracy(cnn_lstm_model,X_train,Y_train,X_test,Y_test)

In [ ]:
#Classification report
report_nn(cnn_lstm_model,X_test,Y_test)

##### Confusion Matrix

In [ ]:
#cnn_lstm_model0=cnn_lstm_model
print('--Confusion Matrix for CNN+LSTM--')
print_confusion(cnn_lstm_model,X_test,Y_test)

## Bidirectional LSTM models

In [ ]:
lstmbd_model = BID_LSTM_model(embedding_matrix,vocab_len,emb_dim)

In [ ]:
%%time
history=lstmbd_model.fit(X_train, np.asarray(Y_train), validation_data=(X_val, np.asarray(Y_val)),epochs=epochs, batch_size=batch_size)

In [ ]:
#plot accuracy
plot_accuracy(history,epochs)

In [ ]:
# Accuracy
train_test_accuracy(cnn_lstm_model,X_train,Y_train,X_test,Y_test)

In [ ]:
#Classification report
report_nn(cnn_lstm_model,X_test,Y_test)

## Bidirectional GRU

In [ ]:
grubd_model = BID_GRU_model(embedding_matrix,vocab_len,emb_dim)

In [ ]:
%%time
history=grubd_model.fit(X_train, np.asarray(Y_train), validation_data=(X_val, np.asarray(Y_val)),epochs=epochs, batch_size=batch_size)

In [ ]:
#plot accuracy
plot_accuracy(history,epochs)

In [ ]:
# Accuracy
train_test_accuracy(grubd_model,X_train,Y_train,X_test,Y_test)

In [ ]:
#Classification report
report_nn(grubd_model,X_test,Y_test)